# Time-varying loadings

**Exchange rates and macroeconomic fundamentals: Evidence of instabilities from time‐varying factor loadings**

Eric Hillebrand, Jakob Guldbæk Mikkelsen, Lars Spreng, Giovanni Urga

In [7]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import numpy as np
from latentfactors.core.tvp import LoglikNomean, KFNomean
from latentfactors import cfg
from scipy.optimize import minimize

k = 3

fx = pd.read_excel(fr"{cfg.fldr}/data/hmsu/fx.xlsx", index_col=0)
eco_us = pd.read_excel(fr"{cfg.fldr}/data/hmsu/usa.xlsx", index_col=0)

# Log-difference FX
dlog_fx = fx.apply(np.log).diff().reindex(eco_us.index).mul(-1)
t, n = dlog_fx.shape

## Factors estimation

In [8]:
# Standardize economic indicators
scaler = StandardScaler()
scaler.fit(eco_us)
X = scaler.transform(eco_us)

# Transform them into actors
np.random.seed(1234)
svd = TruncatedSVD(n_components=k)
svd.fit(X)
V = svd.components_  # Right singular vectors
S = svd.singular_values_

U = X @ V.T @ np.linalg.inv(np.diag(S))  # Left singular values
mF = np.multiply(U, np.sqrt(t))

## Estimate loadings

In [9]:
mBeta = {}

# For-loop
crncy = "AUD"

# OLS Estimates
r = dlog_fx[crncy].values.reshape(-1, 1)  # FX Series
b = np.linalg.inv(mF.T @ mF) @ mF.T @ r  # OLS Beta
r_hat = mF @ b  # OLS predicted values
e = r - r_hat
s_e = e.T @ e / t  # Asymptotic OLS variance
# TODO: Compute White VCV
# TODO: Compute HAC VCV
L_ols = -1/2 * t * np.log(s_e.item())  # Likelihood values for restricted model with all loadings constant
# mBeta[crncy] = b.flatten()

In [10]:
vP0 = np.concatenate([np.zeros((2 * k, 1)), np.log(s_e)])
print(vP0)

opt = minimize(
    fun=LoglikNomean, 
    x0=vP0.flatten(), 
    args=(r.copy(), mF.copy()),
    method="SLSQP",
    options={'maxiter': 5e4, 'maxfev': 6e4, 'disp': True, 'gtol': 1e-10}
)

print(opt)

[[ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [-7.3266721]]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1192.5776914243604
            Iterations: 5
            Function evaluations: 43
            Gradient evaluations: 5
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1192.5776914243604
       x: [ 7.698e-03 -5.020e+01  1.359e-02 -7.121e+01  5.378e-03
           -6.689e+01 -7.327e+00]
     nit: 5
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00 -4.883e-04]
    nfev: 43
    njev: 5


C:\Users\franc\AppData\Local\Temp\ipykernel_17100\461742189.py:4: OptimizeWarning: Unknown solver options: maxfev, gtol
  opt = minimize(


In [11]:
v = [
    0.627649798962410,
    -9.02798243686879,
    0.377221893082486,
    -9.39757147204541,
    0.255891882651142,
    -10.6338108464130,
    -7.89142932448150,
]

LoglikNomean(
    vP=np.array(v),
    Data=r,
    mF=mF
)

-1223.8628864214343

In [12]:
from filterpy.kalman import KalmanFilter
